In [1]:
###Business Understanding

print("This work will give the detailed analysis of Life Expectancy based on some social, health-based and economic factors.")

This work will give the detailed analysis of Life Expectancy based on some social, health-based and economic factors.


In [2]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DataFrame').getOrCreate()

from pyspark.sql import functions as f
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [3]:
### Data Understanding
###Reading the csv file
df = spark.read.csv('datasets_12603_17232_Life Expectancy Data.csv', header=True, inferSchema=True)

In [4]:
###Showing the 1st Row
df.head(1)

[Row(Country='Afghanistan', Year=2015, Status='Developing', Life expectancy =65.0, Adult Mortality=263, infant deaths=62, Alcohol=0.01, percentage expenditure=71.27962362, Hepatitis B=65, Measles =1154,  BMI =19.1, under-five deaths =83, Polio=6, Total expenditure=8.16, Diphtheria =65,  HIV/AIDS=0.1, GDP=584.25921, Population=33736494.0,  thinness  1-19 years=17.2,  thinness 5-9 years=17.3, Income composition of resources=0.479, Schooling=10.1)]

In [5]:
#Head in Python
df.show(5)

+-----------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+----------+-----------+---------------------+-------------------+-------------------------------+---------+
|    Country|Year|    Status|Life expectancy |Adult Mortality|infant deaths|Alcohol|percentage expenditure|Hepatitis B|Measles | BMI |under-five deaths |Polio|Total expenditure|Diphtheria | HIV/AIDS|       GDP| Population| thinness  1-19 years| thinness 5-9 years|Income composition of resources|Schooling|
+-----------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+----------+-----------+---------------------+-------------------+-------------------------------+---------+
|Afghanistan|2015|Developing|            65.0|            263|           62|   

In [6]:
#Column Names
df.columns

['Country',
 'Year',
 'Status',
 'Life expectancy ',
 'Adult Mortality',
 'infant deaths',
 'Alcohol',
 'percentage expenditure',
 'Hepatitis B',
 'Measles ',
 ' BMI ',
 'under-five deaths ',
 'Polio',
 'Total expenditure',
 'Diphtheria ',
 ' HIV/AIDS',
 'GDP',
 'Population',
 ' thinness  1-19 years',
 ' thinness 5-9 years',
 'Income composition of resources',
 'Schooling']

In [7]:
#Data types of Columns
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Status: string (nullable = true)
 |-- Life expectancy : double (nullable = true)
 |-- Adult Mortality: integer (nullable = true)
 |-- infant deaths: integer (nullable = true)
 |-- Alcohol: double (nullable = true)
 |-- percentage expenditure: double (nullable = true)
 |-- Hepatitis B: integer (nullable = true)
 |-- Measles : integer (nullable = true)
 |--  BMI : double (nullable = true)
 |-- under-five deaths : integer (nullable = true)
 |-- Polio: integer (nullable = true)
 |-- Total expenditure: double (nullable = true)
 |-- Diphtheria : integer (nullable = true)
 |--  HIV/AIDS: double (nullable = true)
 |-- GDP: double (nullable = true)
 |-- Population: double (nullable = true)
 |--  thinness  1-19 years: double (nullable = true)
 |--  thinness 5-9 years: double (nullable = true)
 |-- Income composition of resources: double (nullable = true)
 |-- Schooling: double (nullable = true)



In [8]:
df.select('Life expectancy ').show(10)

+----------------+
|Life expectancy |
+----------------+
|            65.0|
|            59.9|
|            59.9|
|            59.5|
|            59.2|
|            58.8|
|            58.6|
|            58.1|
|            57.5|
|            57.3|
+----------------+
only showing top 10 rows



In [9]:
#Showing min, max, avg values of Life Expectancy
df.describe('Life expectancy ').show()

+-------+-----------------+
|summary| Life expectancy |
+-------+-----------------+
|  count|             2928|
|   mean|69.22493169398912|
| stddev|9.523867487824305|
|    min|             36.3|
|    max|             89.0|
+-------+-----------------+



In [10]:
#Counting the Missing Values
def count_missings(spark_df,sort=True):
    df = spark_df.select([f.count(f.when(f.isnan(c) | f.isnull(c), c)).alias(c) for (c,c_type) in spark_df.dtypes if c_type not in ('timestamp', 'string', 'date')]).toPandas()
    if len(df) == 0:
        print("There are no any missing values!")
        return None
    if sort:
        return df.rename(index={0: 'count'}).T.sort_values("count",ascending=False)
    return df

In [11]:
count_missings(df)

,count
Population,652
Hepatitis B,553
GDP,448
Total expenditure,226
Alcohol,194
Income composition of resources,167
Schooling,163
thinness 5-9 years,34
thinness 1-19 years,34
BMI,34


In [12]:
#Showing the Outliers
class Outlier():
    def __init__(self, df):
        self.df = df
    def _calculate_bounds(self):
        bounds = {
            c: dict(zip(["q1", "q3"], self.df.approxQuantile(c, [0.25, 0.75], 0)))
            for c, d in zip(self.df.columns, self.df.dtypes) if d[1] in ["bigint", "double"]
        }
        for c in bounds:
            iqr = bounds[c]['q3'] - bounds[c]['q1']
            bounds[c]['min'] = bounds[c]['q1'] - (iqr * 1.5)
            bounds[c]['max'] = bounds[c]['q3'] + (iqr * 1.5)
        return bounds
    def _flag_outliers_df(self):
        bounds = self._calculate_bounds()
        outliers_col = [f.when(~f.col(c).between(bounds[c]['min'], bounds[c]['max']),f.col(c)).alias(c + '_outlier')
                        for c in bounds]
        return self.df.select(*outliers_col)
    def show_outliers(self):
        outlier_df = self._flag_outliers_df()
        for outlier in outlier_df.columns:
            outlier_df.select(outlier).filter(f.col(outlier).isNotNull()).show(5)


In [13]:
Outlier(df).show_outliers()

+-----------------+
|Schooling_outlier|
+-----------------+
|              0.0|
|              0.0|
|              0.0|
|              0.0|
|              0.0|
+-----------------+
only showing top 5 rows

+-------------+
| BMI _outlier|
+-------------+
+-------------+

+-----------------------------+
| thinness  1-19 years_outlier|
+-----------------------------+
|                         17.2|
|                         17.5|
|                         17.7|
|                         17.9|
|                         18.2|
+-----------------------------+
only showing top 5 rows

+-------------------------+
|Total expenditure_outlier|
+-------------------------+
|                    13.66|
|                    14.39|
|                     12.6|
|                    13.73|
|                    17.24|
+-------------------------+
only showing top 5 rows

+------------------+
|Population_outlier|
+------------------+
|       3.3736494E7|
|       3.1731688E7|
|       2.6616792E7|
|       2.4118

In [14]:
df.count()

2938

In [15]:
#Missing Value Replacement
from pyspark.sql.functions import mean
mean_pop = df.select(mean(df['Population'])).collect()
mean_pop_val = mean_pop[0][0]
df1=df.na.fill(mean_pop_val, subset=['Population'])
mean_hep = df1.select(mean(df['Hepatitis B'])).collect()
mean_hep_val = mean_hep[0][0]
df1=df1.na.fill(mean_hep_val, subset=['Hepatitis B'])
mean_gdp = df1.select(mean(df['GDP'])).collect()
mean_gdp_val = mean_gdp[0][0]
df1=df1.na.fill(mean_gdp_val, subset=['GDP'])
mean_totexp = df1.select(mean(df['Total expenditure'])).collect()
mean_totexp_val = mean_totexp[0][0]
df1=df1.na.fill(mean_totexp_val, subset=['Total expenditure'])
mean_alc = df1.select(mean(df['Alcohol'])).collect()
mean_alc_val = mean_alc[0][0]
df1=df1.na.fill(mean_alc_val, subset=['Alcohol'])
mean_inc = df1.select(mean(df['Income composition of resources'])).collect()
mean_inc_val = mean_inc[0][0]
df1=df1.na.fill(mean_inc_val, subset=['Income composition of resources'])
mean_sch = df1.select(mean(df['Schooling'])).collect()
mean_sch_val = mean_sch[0][0]
df1=df1.na.fill(mean_sch_val, subset=['Schooling'])

In [16]:
df2=df1.na.drop()

In [17]:
count_missings(df2)

,count
Year,0
Life expectancy,0
Income composition of resources,0
thinness 5-9 years,0
thinness 1-19 years,0
Population,0
GDP,0
HIV/AIDS,0
Diphtheria,0
Total expenditure,0


In [18]:
df2.count()

2888

In [19]:
#Creating new Columns
df2=df2.withColumn('Doubled_Life_Expectancy', df2['Life expectancy ']*2)
df2.show(5)

+-----------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+----------+-----------+---------------------+-------------------+-------------------------------+---------+-----------------------+
|    Country|Year|    Status|Life expectancy |Adult Mortality|infant deaths|Alcohol|percentage expenditure|Hepatitis B|Measles | BMI |under-five deaths |Polio|Total expenditure|Diphtheria | HIV/AIDS|       GDP| Population| thinness  1-19 years| thinness 5-9 years|Income composition of resources|Schooling|Doubled_Life_Expectancy|
+-----------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+----------+-----------+---------------------+-------------------+-------------------------------+---------+-----------------------+
|Afghan

In [20]:
#Data Split
data1,data2 = df2.randomSplit([0.5,0.5])
print("First Dataset Count: " + str(data1.count()))
print("Second Dataset Count: " + str(data2.count()))

First Dataset Count: 1448
Second Dataset Count: 1440


In [21]:
#Data Integration
newdata = data2.union(data2)
print("Merged Dataset Count: " + str(newdata.count()))

Merged Dataset Count: 2880


In [22]:
#Changing format of Status Column
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

stringIndexer = StringIndexer(inputCol="Status", outputCol="Status_encoded")
model = stringIndexer.fit(df2)
indexed = model.transform(df2)
encoder = OneHotEncoder(dropLast=False, inputCol="Status_encoded", outputCol="Status_vec")
encoded = encoder.transform(indexed)
encoded.show(2)

+-----------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+----------+-----------+---------------------+-------------------+-------------------------------+---------+-----------------------+--------------+-------------+
|    Country|Year|    Status|Life expectancy |Adult Mortality|infant deaths|Alcohol|percentage expenditure|Hepatitis B|Measles | BMI |under-five deaths |Polio|Total expenditure|Diphtheria | HIV/AIDS|       GDP| Population| thinness  1-19 years| thinness 5-9 years|Income composition of resources|Schooling|Doubled_Life_Expectancy|Status_encoded|   Status_vec|
+-----------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+----------+-----------+---------------------+-------------------+----------------

In [23]:
#Transform the data
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors

assembler = VectorAssembler(
  inputCols=['Year',
             'Adult Mortality',
             'infant deaths',
             'Alcohol',
             'percentage expenditure',
             'Hepatitis B',
             'Measles ',
             ' BMI ',
             'under-five deaths ',
             'Polio',
             'Total expenditure',
             'Diphtheria ',
             ' HIV/AIDS',
             'GDP',
             'Population',
             ' thinness  1-19 years',
             ' thinness 5-9 years',
             'Income composition of resources',
             'Schooling'],
              outputCol="features")
final=assembler.transform(df2)

normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)
l1NormData = normalizer.transform(final)
print("Normalized using L^1 norm")
l1NormData.head(1)

Normalized using L^1 norm


[Row(Country='Afghanistan', Year=2015, Status='Developing', Life expectancy =65.0, Adult Mortality=263, infant deaths=62, Alcohol=0.01, percentage expenditure=71.27962362, Hepatitis B=65, Measles =1154,  BMI =19.1, under-five deaths =83, Polio=6, Total expenditure=8.16, Diphtheria =65,  HIV/AIDS=0.1, GDP=584.25921, Population=33736494.0,  thinness  1-19 years=17.2,  thinness 5-9 years=17.3, Income composition of resources=0.479, Schooling=10.1, Doubled_Life_Expectancy=130.0, features=DenseVector([2015.0, 263.0, 62.0, 0.01, 71.2796, 65.0, 1154.0, 19.1, 83.0, 6.0, 8.16, 65.0, 0.1, 584.2592, 33736494.0, 17.2, 17.3, 0.479, 10.1]), normFeatures=DenseVector([0.0001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999, 0.0, 0.0, 0.0, 0.0]))]

In [24]:
train_data,test_data = final.randomSplit([0.8,0.2])

In [25]:
print("Training Dataset Count: " + str(train_data.count()))
print("Testing Dataset Count: " + str(test_data.count()))

Training Dataset Count: 2323
Testing Dataset Count: 565


In [26]:
train_data.show(2)
test_data.show(2)

+-----------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+---------+----------+---------------------+-------------------+-------------------------------+---------+-----------------------+--------------------+
|    Country|Year|    Status|Life expectancy |Adult Mortality|infant deaths|Alcohol|percentage expenditure|Hepatitis B|Measles | BMI |under-five deaths |Polio|Total expenditure|Diphtheria | HIV/AIDS|      GDP|Population| thinness  1-19 years| thinness 5-9 years|Income composition of resources|Schooling|Doubled_Life_Expectancy|            features|
+-----------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+---------+----------+---------------------+-------------------+-------------------------------+------

In [27]:
#Linear Regression on Raw data
lr1 = LinearRegression(featuresCol='features', labelCol='Life expectancy ', predictionCol='prediction')
lrModel1 = lr1.fit(final)

In [28]:
print("Coefficients: {}".format(str(lrModel1.coefficients)))
print('\n')
print("Intercept:{}".format(str(lrModel1.intercept)))

Coefficients: [-0.016775066480263216,-0.02021463495627747,0.09703734635663123,0.11226934549979334,0.00011686650319924771,-0.013389490301605158,-2.1006424290000457e-05,0.039221008734863685,-0.0724829388458426,0.02848430572737853,0.07313736643514569,0.039460206277326416,-0.470214588989915,3.604485157410914e-05,1.3546834910560836e-11,-0.07249921981027509,-0.0018471591129023345,6.076345390213576,0.7474028508588368]


Intercept:87.43174961867935


In [29]:
trainingSummary1 = lrModel1.summary
trainingSummary1.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
|  4.334699850209262|
|-1.8814604164303574|
| -1.954627769416085|
| -2.341008364247223|
|-2.1566593640738603|
+-------------------+
only showing top 5 rows



In [30]:
#Printing Root Mean Squared Error 
print("RMSE: {}".format(trainingSummary1.rootMeanSquaredError))

#Printing R-Squared
print("R^2: {}".format(trainingSummary1.r2))


RMSE: 4.035431133886131
R^2: 0.8193241942414985


In [31]:
#Linear Regression on Cleaned data
lr = LinearRegression(labelCol='Life expectancy ')
lrModel = lr.fit(train_data)

In [32]:
print("Coefficients: {}".format(str(lrModel.coefficients))) 
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coefficients: [-0.010370962209208238,-0.020557962781258935,0.09546007513215636,0.1305270043241027,0.00011344715003270555,-0.014688527678440377,-2.4054212439604347e-05,0.043084716998849956,-0.07110743268663808,0.037389463824023646,0.05744539102890746,0.03825636633825681,-0.4553035040064571,3.7969965060886336e-05,-4.903313326668561e-10,-0.09474501712830705,0.010878880255708497,6.235564387733529,0.6798674395176199]


Intercept:74.67749373936209


In [33]:
test_results = lrModel.evaluate(test_data)
test_results.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
|  3.553915682907416|
|-1.0050629591586002|
|-1.4663184595800445|
|-1.8349222109651606|
|-1.1339297986971104|
+-------------------+
only showing top 5 rows



In [34]:
print("RSME: {}".format(test_results.rootMeanSquaredError))
print("R^2: {}".format(test_results.r2))

RSME: 3.9017959355131024
R^2: 0.8267240736551594


In [35]:
#Random Forest Model
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(featuresCol="features", labelCol="Life expectancy ", predictionCol="Prediction", seed=42)

In [36]:
model = rf.fit(train_data)
predictions = model.transform(test_data)

In [37]:
predictions.select("Prediction", "Life expectancy ").show(5)

+------------------+----------------+
|        Prediction|Life expectancy |
+------------------+----------------+
|59.858369039632905|            57.0|
|59.858369039632905|            57.3|
| 60.39316722383544|            58.6|
| 61.36146317833932|            59.2|
| 74.56249939466832|            73.0|
+------------------+----------------+
only showing top 5 rows



In [38]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="Life expectancy ", predictionCol="Prediction")
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

#Printing Model Metrics
print('RMSE: ' + str(rmse))
print('R^2: ' + str(r2))

RMSE: 2.467552991530858
R^2: 0.9306985851134318
